In [1]:
# Import dependencies 
import pandas as pd
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# Set the column width to view the text message data.
pd.set_option('max_colwidth', 200)


/Users/pranavreddy/anaconda3/envs/dev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def sms_classification(sms_text_df):
    # Set the features variable to the text message column.
    X = sms_text_df['text_message']
    # Set the target variable to the "label" column.
    y = sms_text_df['label']

    # Split data into training and testing and set the test_size = 33%
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

    # Build a pipeline to transform the test set to compare to the training set.
    text_clf = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')), ('clf', LinearSVC()),])

    # Fit the model to the transformed training data and return model.
    return text_clf.fit(X_train, y_train)
     

In [3]:
sms_text_df = pd.read_csv("./Resources/SMSSpamCollection.csv")
sms_text_df.head()

,label,text_message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [4]:
# Call the sms_classification function with the DataFrame and set the result to the "text_clf" variable
text_clf = sms_classification(sms_text_df)

/Users/pranavreddy/anaconda3/envs/dev/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [5]:
# Function takes in the SMS text and predicts the whether the text is "not spam" or "spam". 
def sms_prediction(text):
    result = text_clf.predict([text])
    if result[0] == "ham": 
        return f'The text message: {text}, is not spam.'
    else: 
        return f'The text message: {text}, is spam.'
    

In [6]:
# Define a gradio interface with textbox input and output.  
app = gr.Interface(fn=sms_prediction, 
                   inputs=[gr.Textbox(label="What message would you like to check for spam? ")], 
                   outputs=gr.Textbox(label="Classified as:"))
    
# Launch the app.
app.launch(share=True, show_error=True)

Running on local URL:  http://127.0.0.1:7865
Running on public URL: https://1a93f36d42c5da0bc3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
